In [1]:
import numpy as np
%matplotlib inline
from matplotlib import pyplot as plt
from scipy.optimize import minimize
from scipy.optimize import least_squares, fsolve

In [2]:
R = 8.3144626 #J/mol-K
F = 96485.33212 #C/mol

# Air electrode = cathode
O2 + 2H20 +4e- = 2OH-
# Zn electrode = anode
Zn + 4OH- = Zn(OH)4^2- +2e-

### Butler Volmer equations to describe kinetics
    i_0 = exchange current density - the value of the current density at equilibrium
    alpha_a = anodic charge transfer coefficient
    alpha_c = cathodic charge transfer coefficient
    alpha_a + alpha_c = number of electrons transferred in reaction
    eta = surface overpotential = potential - equilibrium potential

In [3]:
# General Butler-Volmer equation
def bv_eq(i_0, alpha_a, alpha_c, eta):
    R = 8.3144626
    T = 273 #K
    i = i_0*(np.exp((alpha_a*F*eta)/(R*T)) - np.exp((-alpha_c*F*eta)/(R*T)))
    return i

In [24]:
# Implement one version of BV equation for each reaction
# Model parameters for oxidation reduction reaction at air electrode
i_0_air = 1 #A 
alpha_a_air = 2 #good approximation 0.5*4 electrons #unitless
alpha_c_air = 2 #good approximation 0.5*4 electrons #unitless

# Model parameters for zinc to zincate reaction at air electrode
i_0_zn = 1 #A 
alpha_a_zn = 1 #good approximation 0.5*4 electrons #unitless
alpha_c_zn = 1 #good approximation 0.5*4 electrons #unitless

# Equilibrium Potential
U_air = 1.2 #V
U_zn = -1 #V
U_cell = U_air - U_zn #V doublecheck - also should be pos or neg?


# Need to solve this system of equations??
# need to initialize etas?
# Cell potential
#V_cell = U_cell - eta_air - eta_zn

# Calculate current
# CURRENTS MUST EQUAL EACH OTHER - THAT HOW SYSTEM IS FULLY SPECIFIED

#i_air = bv_eq(i_o_air, alpha_a_air, alpha_c_air, eta_air)
#i_zn = bv_eq(i_o_zn, alpha_a_zn, alpha_c_zn, eta_zn)
#i_air = i_zn

def current(X, *data):
    eta_air = X #eta_zn = X
    eta_zn = .5
    i_0_air, alpha_a_air, alpha_c_air, i_0_zn, alpha_a_zn, alpha_c_zn, R, T, U_cell, P = data
    i_air = i_0_air*(np.exp((alpha_a_air*F*eta_air)/(R*T)) - np.exp((-alpha_c_air*F*eta_air)/(R*T)))
    i_zn = i_0_zn*(np.exp((alpha_a_zn*F*eta_zn)/(R*T)) - np.exp((-alpha_c_zn*F*eta_zn)/(R*T)))
    
    V_cell = U_cell - eta_air - eta_zn
    
    #i_air = bv_eq(i_o_air, alpha_a_air, alpha_c_air, eta_air)
    #i_zn = bv_eq(i_o_zn, alpha_a_zn, alpha_c_zn, eta_zn)
    return [i_air - i_zn, P - i_air*A*V_cell]

'''
def func2(X, *data):
    U_cell = data
    return U_cell - V_cell - eta_air - eta_zn
'''

T = 273 #K
P = 500
data = (i_o_air, alpha_a_air, alpha_c_air, i_o_zn, alpha_a_zn, alpha_c_zn, R, T, U_cell, P)
#eta_air_0, eta_zn_0 = 0, 0
#guess = [eta_air_0, eta_zn_0]
guess = 1
sol = fsolve(current, guess, args=data)
print(sol)
# Area
A = 1 #units? m^2?
#I = A*i

#Power
#P = I*V_cell


ValueError: object too deep for desired array

error: Result from function call is not a proper array of floats.

In [10]:
fsolve?